# L5c: A Deeper Dive into Flux Balance Analysis Constraints
In this lecture, we will continue our disucssion of flux balance analysis, and will explore the what we can do with the material balance and flux bounds constraints. However be before we do that, let's review the key concepts we'll talk about today and the linear programming problem that we will solve.

* __Flux balance analysis (FBA)__ is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where production, consumption and transport rates of metabolites are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. 
* __Material balance constraints__ are used to ensure that the flow of metabolites into and out of a system is balanced (and physical). These constraints are typically represented as a set of linear equations, where the coefficients represent the stoichiometry of the reactions in the system, in combination with transport and dilution terms.
* __Flux bounds constraints__ limit the range of possible fluxes through a metabolic network. These bounds can be used to incorporate additional information into the FBA problem, such as experimental data or prior knowledge about the system.


Lecture notes can be downloaded: [here!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5c/docs/Notes.pdf)

## Flux Balance Analysis Problem Formulation
Flux balance analysis (FBA) is a mathematical approach used to analyze the flow of metabolites through a metabolic network. It assumes a steady state where production, consumption and transport rates of metabolites are balanced. The FBA problem is formulated as a linear programming (LP) problem to maximize or minimize fluxes through the network, subject to constraints. The linear program is:
$$
\begin{align*}
\max_{\hat{v}}\quad&  \sum_{i\in\mathcal{R}}c_{i}\hat{v}_{i}\\
\text{subject to}\quad & \sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s} + \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V = \frac{d}{dt}\left(C_{i}V\right)\qquad\forall{i\in\mathcal{M}}\\
& \mathcal{L}_{j}\leq\hat{v}_{j}\leq\mathcal{U}_{j}\qquad\forall{j\in\mathcal{R}}
\end{align*}
$$
Here, $\sigma_{ij}$ are elements of the stoichiometric matrix $\mathbf{S}$, $c_{i}$ are objective coefficients (you choose), $\hat{v}_{i}$ are unknown fluxes, and $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ are the lower and upper flux bounds, respectively.

## A model for flux bounds
The flux bounds are important constraints in flux balance analysis calculations and the convex decomposition of the stoichiometric array. Beyond their role in the flux estimation problem, the flux bounds are _integrative_, i.e., these constraints integrate many types of genetic and biochemical information into the flux estimation problem. A general model for these bounds is given by:
$$
\begin{align}
-\delta_{j}\left[{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}\right]\leq{v_{j}}\leq{V_{max,j}^{\circ}}\left(\frac{e}{e^{\circ}}\right)\theta_{j}\left(\dots\right){f_{j}\left(\dots\right)}
\end{align}
$$
where $V_{max,j}^{\circ}$ denotes the maximum reaction velocity (units: `flux`) computed at some _characteristic enzyme abundance_ $e^{\circ}$ (units: `concentration`), and $e$ is the enzyme abundance (units: `concentration`). The term $\left(e/e^{\circ}\right)$ is a correction for enzyme abundance (units: `dimensioness`). The $\theta_{j}\left(\dots\right)\in\left[0,1\right]$ is the fraction of maximial enzyme activity (a function or measured with units: `dimensionless`), i.e., it describes [allosteric](https://en.wikipedia.org/wiki/Allosteric_regulation) effects on the reaction rate.
Finally, the $f_{j}\left(\dots\right)$ is a function describing the substrate dependence of the reaction rate $j$ (units: `dimensionless`). 

* __Parameters__: Both the $\theta_{j}\left(\dots\right)$ and $f_{j}\left(\dots\right)$ models can have associated parameters, e.g., saturation or binding constants, etc. Thus, we need to estimate these from literature studies or experimental data.

### Reversibility
The binary parameter $\delta_{j}\in\left\{0,1\right\}$ is the reversibility parameter. The value of $\delta_{j}$ is determined by the reversibility of reaction $j$: 
* __Values of δⱼ__: If reaction $j$ is __reversible__ $\delta_{j}=1$ or, If reaction $j$ is __irreversible__ $\delta_{j}=0$

The reversibility parameter can be computed in one of several possible ways. For example, one method in the literature is to use the sign of Gibbs reaction energy:

* if $\text{sign}\left(\Delta{G}^{\circ}\right)<0$ then $\delta = 0$ (irreversible)
* if $\text{sign}\left(\Delta{G}^{\circ}\right)>0$ then $\delta = 1$ (reversible)

Alternatively, the value of $\delta$ can be assigned based upon a cutoff on the equilibrium constant:

* if $K_{eq}>\star$ then $\delta = 0$ (irreversible)
* if $K_{eq}\leq\star$ then $\delta = 1$ (reversible)

where the metabolic engineer specifies the value $\star$.

### The eQuilibrator application programming interface (API)

The [eQuilibrator application programming interface](https://equilibrator.weizmann.ac.il) is a tool for thermodynamic calculations in biological reaction networks. It was developed by the [Milo lab](https://www.weizmann.ac.il/plants/Milo/) at the Weizmann Institute in Rehovot, Israel.

* [Beber ME, Gollub MG, Mozaffari D, Shebek KM, Flamholz AI, Milo R, Noor E. eQuilibrator 3.0: a database solution for thermodynamic constant estimation. Nucleic Acids Res. 2022 Jan 7;50(D1): D603-D609. doi: 10.1093/nar/gkab1106. PMID: 34850162; PMCID: PMC8728285.](https://pubmed.ncbi.nlm.nih.gov/34850162/)

The [`eQuilibrator.jl` package](https://github.com/stelmo/eQuilibrator.jl) is a [Julia](https://julialang.org) wrapper around eQuilibrator (which is written in Python). 